# Initial Setup

## 📬 Setting Up Gmail API Access

To enable your AI agent to read and send emails, you'll need to set up the Gmail API on your Google account. Follow these steps carefully:

- 🔑 Step 1: Open Google Cloud Console
  - Go to [https://console.cloud.google.com/](https://console.cloud.google.com/)
  - Sign in using the **Google account** you want the agent to access

- 🆕 Step 2: Create a New Project
  - Click the project dropdown (top nav bar)
  - Click **"New Project"**, give it a name, and click **"Create"**

- ⚙️ Step 3: Enable Gmail API
  - Navigate to **APIs & Services > Library**
  - Search for **Gmail API**
  - Click on it, then click **"Enable"**

- 🔐 Step 4: Configure OAuth Consent Screen
  - Go to **APIs & Services > OAuth consent screen**
  - Choose **"External"**
  - Fill in required fields (app name, user support email, etc.)
  - Add Gmail API scopes:

- 🧾 Step 5: Create OAuth 2.0 Credentials
  - Go to **APIs & Services > Credentials**
  - Click **"Create Credentials" → "OAuth Client ID"**
  - Choose **Desktop App**
  - Download the `credentials.json` file
  - Store the json file at `/content/drive/MyDrive/email_assistant/`

🔁 Save this file in the same directory as your notebook. We'll use it to authenticate Gmail access.

## 💡 Setting Up Gemini API Access (Google AI Studio)

To generate intelligent email replies, we'll use Google's Gemini model via the Generative Language API.

- 🔗 Step 1: Visit Google AI Studio
  - Go to [https://ai.google.dev/](https://ai.google.dev/)
  - Sign in with your Google account

- 📋 Step 2: Create a New API Key
  - Click on **"Get API Key"**
  - Accept the terms and select your project
  - Copy the generated **API key**

- 🗝️ Step 3: Paste the API Key
  - You’ll need to `paste the Gemini API Key` when prompted

# Dependancies

## Install Required Libraries

- 📦 Installing Required Python Libraries
  - **Gmail API access** via `google-api-python-client`, `google-auth-httplib2`, and `google-auth-oauthlib`
  - **Gemini LLM access** via `google-generativeai`

✅ Run this cell once to set up your environment. If you're running in Colab or a fresh environment, this step is required.

In [ ]:
# Gmail API libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

# Gemini LLM SDK from Google AI
!pip install --upgrade google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 89.9 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.164.0
    Uninstalling google-api-python-client-2.164.0:
      Successfully uninstalled google-api-python-client-2.164.0


## Mounting Google Drive

This step mounts your Google Drive to the Colab environment so you can:

- Load sensitive files like `credentials.json` from a secure location
- Save your `token.pickle` after authenticating with Gmail
- Store logs, results, or exports if needed

In [ ]:
# Mount Google Drive to access credentials and store tokens
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing Required Libraries

This section imports all the core Python libraries used in this project:

- `os`, `pickle`, `json`: File handling and serialization
- `google_auth_oauthlib`, `googleapiclient`: For Gmail API authentication and access
- `google.generativeai`: Gemini LLM SDK
- `getpass`: For secure input if needed
- `time`, `random`, `re`: For rate-limiting, delays, and regular expression handling
- `google.api_core.exceptions.TooManyRequests`: To handle Gemini API rate limit errors


In [ ]:
# 🗂 File and system operations
import os
import pickle
import json

# 🔐 Google OAuth & Gmail API
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build

# 🔑 Secure input (e.g., API key prompt)
import getpass

# 🤖 Gemini LLM SDK
import google.generativeai as genai

# 🕒 Retry and control logic
import time       # for backoff delays
import random     # for jitter
import re         # for placeholder and response parsing

# 🚨 Gemini API-specific exception
from google.api_core.exceptions import TooManyRequests

# 📧 Email message construction & decoding
from email.mime.text import MIMEText
from email import message_from_bytes
import base64

# ✉️ Gmail Client Class

This class manages all interactions with the Gmail API, including:

- Authenticating the user with OAuth 2.0 (using `credentials.json`)
- Fetching `unread emails` from the inbox
- Extracting `plain text content` from email messages
- Getting the sender's email address
- `Sending emails` using MIME formatting
- `Marking emails as read` after replying

> 🔒 Note: The access token is stored in `token.pickle` after first-time authorization to avoid repeated manual login.

In [ ]:
# --- Gmail Client Class ---
class GmailClient:
    def __init__(self, creds_path, token_path):
        self.creds_path = creds_path
        self.token_path = token_path
        self.service = self.authenticate()

    def authenticate(self):
        # ✅ Check if token already exists (reuse it to skip re-auth)
        if os.path.exists(self.token_path):
            with open(self.token_path, 'rb') as token:
                creds = pickle.load(token)
        else:
            # 🔐 Begin OAuth 2.0 flow
            flow = Flow.from_client_secrets_file(
                self.creds_path,
                scopes=["https://www.googleapis.com/auth/gmail.modify"],
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )
            auth_url, _ = flow.authorization_url(prompt='consent')
            print("🔗 Go to this URL, authorize, and paste the code below:")
            print(auth_url)
            code = input("Paste your code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

            # 💾 Save token for future sessions
            with open(self.token_path, 'wb') as token:
                pickle.dump(creds, token)

        # ✅ Build Gmail API service client
        return build('gmail', 'v1', credentials=creds)

    def get_unread_emails(self):
        # 📥 Get unread messages in the inbox
        results = self.service.users().messages().list(userId='me', labelIds=['INBOX', 'UNREAD']).execute()
        messages = results.get('messages', [])
        # Fetch full message details for each
        return [self.service.users().messages().get(userId='me', id=msg['id']).execute() for msg in messages]

    def extract_body(self, message):
        # 🧾 Extract plain text from the message body
        payload = message['payload']
        parts = payload.get('parts', [])
        for part in parts:
            if part['mimeType'] == 'text/plain':
                data = base64.urlsafe_b64decode(part['body']['data']).decode()
                return data
        return "[No text/plain part found]"

    def get_sender(self, message):
        # 📬 Extract the 'From' field from email headers
        headers = message['payload']['headers']
        for h in headers:
            if h['name'].lower() == 'from':
                return h['value']
        return "Unknown"

    def send_email(self, to, subject, body):
        # 📤 Create and send a MIME email
        message = MIMEText(body)
        message['to'] = to
        message['subject'] = subject
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        self.service.users().messages().send(userId='me', body={'raw': raw}).execute()

    def mark_as_read(self, msg_id):
        # ✅ Remove 'UNREAD' label to mark the email as processed
        self.service.users().messages().modify(userId='me', id=msg_id, body={'removeLabelIds': ['UNREAD']}).execute()

# 🤖 LLM Responder Class

This class is responsible for generating intelligent reply suggestions using a large language model (LLM) such as Gemini.

### Key Features:
- Builds `dynamic prompts` using the incoming email body
- Uses `prompt engineering` to ensure polite, structured replies
- Extracts sender's first name and ends replies with a custom signature
- Handles `429 TooManyRequests` errors with exponential backoff retries
- Cleans up the generated text and truncates anything beyond the signature

✨ This class helps you modularly integrate any LLM API into your email agent workflow.

In [ ]:
# --- LLM Responder Class ---
class LLMResponder:
    def __init__(self, model, signature="Rohit Akole\nData Analyst | AI Enthusiast"):
        self.model = model
        self.signature = signature

    def safe_generate_with_retry(self, generate_fn, max_attempts=3):
        # 🚨 Retry logic for handling Gemini's 429 rate limit
        for attempt in range(max_attempts):
            try:
                return generate_fn()
            except TooManyRequests:
                wait = 2 ** attempt + random.uniform(0, 1)
                print(f"⚠️ Rate limited. Retrying in {wait:.2f}s...")
                time.sleep(wait)
        raise Exception("Rate limit exceeded")

    def generate_replies(self, email_body, num_options=3):
        # 🧠 Build the prompt for the LLM
        prompt = (
            f"The following is an email that needs a reply:\n\n{email_body.strip()}\n\n"
            f"Generate {num_options} professional, friendly email replies."
            f"If the sender's name is identifiable, begin the reply with a friendly salutation using their name."
            f"If sender's name is not available, omit the greeting entirely without using a placeholder."
            f"End each reply with this signature:\n\n{self.signature}\n"
            f"Format:\nSubject: ...\nBody: ..."
        )

        # 🔁 Internal function for calling the LLM
        def call():
            response = self.model.generate_content(prompt)
            return response.text.strip().split("Subject:")[1:]

        # ⚙️ Get replies with retry logic if needed
        blocks = self.safe_generate_with_retry(call)

        reply_options = []
        for block in blocks:
            lines = block.strip().splitlines()
            subject = lines[0].strip()
            remaining = "\n".join(lines[1:]).strip()

            # Remove 'Body:' label if present
            if remaining.lower().startswith("body:"):
                remaining = remaining[5:].strip()

            # ✂️ Truncate anything after signature
            if self.signature in remaining:
                body = remaining.split(self.signature)[0].strip() + f"\n{self.signature}"
            else:
                body = remaining.strip()

            reply_options.append({"subject": subject, "body": body})
        return reply_options

# 🤝 Email Agent Controller Class

This class acts as the high-level controller that coordinates between the Gmail client and the LLM responder.

### Key Responsibilities:
- Retrieve unread emails from the inbox
- Extract sender and body content
- Use the LLM to `generate 3 intelligent reply` suggestions
- Allow the user to:
  - Select a `reply option` (1, 2, or 3)
  - Manually `edit` the reply
  - Skip the email
  - Exit the workflow
- Send the selected reply and mark the email as read
- Add delay to avoid Gemini API rate limits

🧠 This class puts the **human in control** while still leveraging AI to speed up reply generation.


In [ ]:
# --- Email Agent Controller Class ---
class EmailAgent:
    def __init__(self, gmail_client: GmailClient, responder: LLMResponder):
        self.gmail = gmail_client # Handles email fetching, sending, marking
        self.llm = responder      # Handles LLM prompt and response

    def run(self):
        # 📥 Fetch unread emails from inbox
        emails = self.gmail.get_unread_emails()
        if not emails:
            print("📭 No unread emails found.")
            return

        for email in emails:
            msg_id = email['id']
            body = self.gmail.extract_body(email)
            sender = self.gmail.get_sender(email)

            # 🔎 Show brief preview of email
            print(f"\n📨 From: {sender}")
            print(f"📝 Body:\n{body[:500]}...\n")  # Limit preview

            # 🤖 Generate 3 suggested replies using the LLM
            replies = self.llm.generate_replies(body, num_options=3)
            if not replies:
                print("⚠️ Failed to generate replies. Skipping.")
                continue

            # 📄 Display reply options
            for i, opt in enumerate(replies, 1):
                print(f"\n--- Option {i} ---")
                print(f"Subject: {opt['subject']}")
                print(f"Body:\n{opt['body']}\n")

            # 🎛️ Prompt user to choose what to do
            print("Choose a reply to send (1, 2, 3), or type 'edit/ed', 'skip/s', or 'exit/e':")
            action = input("→ ").strip().lower()

            # ⏭️ Skip or exit
            if action in ['skip', 's']:
                print("⏭️ Skipping this email.")
                continue
            if action in ['exit', 'e']:
                print("👋 Exiting.")
                break

            # ✅ Select one of the LLM-generated replies
            elif action in ['1', '2', '3']:
                selected = replies[int(action) - 1]
                subject, body = selected['subject'], selected['body']

            # ✏️ Manual edit mode
            elif action in ['edit', 'ed']:
                print("✏️ Enter subject:")
                subject = input("→ ").strip()
                print("📝 Enter body (end with a line containing only 'DONE'):")
                lines = []
                while True:
                    line = input()
                    if line.strip().lower() == 'done':
                        break
                    lines.append(line)
                body = "\n".join(lines).strip()
            else:
                print("❌ Invalid input. Skipping.")
                continue

            # 🔍 Final confirmation before sending
            print(f"\n📤 Final Preview:\nSubject: {subject}\nBody:\n{body}\n")
            print("Send this reply? ['YES / Y', 'NO / N']:\n")
            confirm = input("→ ").strip().lower()

            if confirm in ['yes', 'y']:
                self.gmail.send_email(sender, subject, body)
                self.gmail.mark_as_read(msg_id)
                print("✅ Email sent and marked as read.")
            else:
                print("🚫 Email not sent.")

            # ⏱️ Prevent rate limit when looping through emails
            time.sleep(4)

# 🔐 Configuring Gemini API Access

This cell securely prompts the user for their **Gemini API key** and uses it to configure the `google.generativeai` SDK. This is necessary for generating intelligent email replies via Google's large language models.

🛡️ Your API key is not displayed in plain text for security purposes. It must have access to the Generative Language API on [https://ai.google.dev](https://ai.google.dev).


In [ ]:
# 🔐 Prompt user securely for Gemini API key (input hidden)
gemini_api_key = getpass.getpass("🔐 Enter your Gemini API key: ")

# ⚙️ Configure the Gemini client using the provided key
genai.configure(api_key=gemini_api_key)

🔐 Enter your Gemini API key: ··········


# ⚡ Initializing Gemini Model

This cell initializes the Gemini model that will be used to generate intelligent email replies.

We're using:
- `gemini-2.0-flash` - a fast, lightweight version ideal for real-time applications

You can also switch to `gemini-pro` if you want more nuanced replies at the cost of speed.

In [ ]:
# ⚙️ Instantiate Components
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

# ✉️ Initialize Gmail Client

This cell creates an instance of the `GmailClient` class, which handles:

- Gmail API authentication using your `credentials.json` and `token.pkl` files
- Fetching **unread emails, sending replies,** and **marking messages as read**

🔁 If this is your first time running the notebook, you'll be prompted to authorize access to your Gmail account.

In [ ]:
# ✉️ Initialize Gmail client with credentials and token from Google Drive
gmail = GmailClient(
    creds_path='/content/drive/MyDrive/email_assistant/credentials.json',
    token_path='/content/drive/MyDrive/email_assistant/token.pkl'
)

# 🤖 Create and Run the AI Email Agent

This final step brings all components together:

- `LLMResponder` is initialized using the Gemini model
- `EmailAgent` is created using the Gmail and LLM components
- `agent.run()` launches the interactive session:
    - Reads unread emails
    - Suggests 3 AI-generated replies
    - Lets the user pick, edit, skip, or exit
    - Sends the chosen reply and marks the email as read

🟢 Run this cell to start your AI-powered email assistant!


In [ ]:
# 🤖 Initialize the LLM responder with the Gemini model
responder = LLMResponder(model)

# 🔁 Create the email agent using Gmail and LLM
agent = EmailAgent(gmail, responder)

# 🚀 Start the interactive email reply session
agent.run()


📨 From: Ananya Shrivastava <ananya.shrivastava.98@gmail.com>
📝 Body:
Hi Professor,

I cannot turn in my homework in time this week. My cat ate my assignment.
Hes a brat and eats everything that is important to me. Last night it ate
my pizza, drank my beer and when he still was hungry he proceeded to EAT
the 20 pages long essay on why "We should be more scared of cats than AI"

I am sorry, I swear this isnt an excuse!

Regards,
Rob C
...


--- Option 1 ---
Subject: Regarding Your Assignment
Body:
Hi Rob,

Thanks for letting me know about the... *ahem*... situation. I understand things can happen, especially with mischievous pets around.

Let's focus on getting this assignment completed. Could you please resubmit the essay by [Suggest a specific date/time]? I'm happy to discuss any challenges you're facing in rewriting it. Perhaps we can brainstorm some approaches to the topic, and maybe even find a way to deter your cat from future scholarly snack attacks.
Rohit Akole
Data Analyst | AI

# 🚀 Further Enhancements (Implemented)

1. **📎 Attachment Support**  
   Automatically downloads, reads, and extracts meaningful content from attachments such as PDF, DOCX, TXT, CSV, and XLSX files.

2. **🔥 Email Prioritization by Urgency & Importance**  
   Uses a custom language model prompt to rank unread emails so that high-impact messages are addressed first.

3. **💬 Sentiment Detection (Body + Attachment)**  
   Detects tone across both the email body and its attachments — helpful for quickly sensing urgency or emotional undertones.

4. **📝 Email + Attachment Summarization**  
   Generates concise summaries from long email threads and embedded documents, improving readability and reducing time spent on context switching.

5. **🤖 AI-Powered Reply Generation with Smart Placeholders**  
   Suggests 3 professional and contextual responses with intelligent placeholders, allowing users to easily personalize before sending.

